<a href="https://colab.research.google.com/github/kuds/rl-atari-pong/blob/main/%5BAtari%20Pong%5D%20Single-Agent%20Reinforcement%20Learning%20DQN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [Atari Tennis] Single-Agent Reinforcement Learning DQN

In [ ]:
!pip install swig

In [ ]:
!pip install gymnasium gymnasium[atari] autorom

In [ ]:
!AutoROM --accept-license

In [ ]:
!pip install stable-baselines3

In [ ]:
import platform
import torch
import numpy
import stable_baselines3
from stable_baselines3 import DQN
from stable_baselines3.common.env_util import make_atari_env
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.callbacks import CallbackList
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.callbacks import CheckpointCallback
from stable_baselines3.common.vec_env import VecVideoRecorder
from stable_baselines3.common.vec_env import VecTransposeImage
import matplotlib.pyplot
import matplotlib
import os
import gymnasium
from importlib.metadata import version
from datetime import datetime
import csv
import time
import google.colab.drive

In [ ]:
print(f"Python Version: {platform.python_version()}")
print(f"Torch Version: {version('torch')}")
print(f"Is Cuda Available: {torch.cuda.is_available()}")
print(f"Cuda Version: {torch.version.cuda}")
print(f"Gymnasium Version: {version('gymnasium')}")
print(f"Numpy Version: {version('numpy')}")
print(f"Stable Baselines3 Version: {version('stable_baselines3')}")

Python Version: 3.10.12
Torch Version: 2.4.1+cu121
Is Cuda Available: True
Cuda Version: 12.1
Gymnasium Version: 0.29.1
Numpy Version: 1.26.4
Stable Baselines3 Version: 2.3.2


In [ ]:
gdrive_path = "/content/gdrive"
google.colab.drive.mount(gdrive_path, force_remount=True)

In [ ]:
rl_type = "DQN"
env_str = "ALE/Tennis-v5"
trained_model = "final_atari_tennis_dqn"
log_dir = "{}/MyDrive/Finding Theta/logs/{}/{}".format(gdrive_path,
                                                       env_str,
                                                       rl_type)
training_data_path = os.path.join(log_dir, "training jobs")
time_folder = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
model_folder_path = os.path.join(log_dir, "training jobs", time_folder)

In [ ]:
#Create Folders
os.makedirs(log_dir, exist_ok=True)
os.makedirs(training_data_path, exist_ok=True)
os.makedirs(model_folder_path, exist_ok=True)

In [ ]:
env = make_atari_env(env_str, n_envs=1, seed=0)
print("Observation Space Size: ", env.observation_space.shape)
print('Actions Space: ', env.action_space)
env.close()

In [ ]:
env = gymnasium.make(env_str)
print("Observation Space Size: ", env.observation_space.shape)
print('Actions Space: ', env.action_space)
env.close()

In [ ]:
# Evaluation Environment Wrapper Arguments
# Disable Frame Skipping
# https://danieltakeshi.github.io/2016/11/25/frame-skipping-and-preprocessing-for-deep-q-networks-on-atari-2600-games/
# https://stable-baselines3.readthedocs.io/en/master/common/atari_wrappers.html
env_val_wrap_args = {"frame_skip": 0, "noop_max": 30}

In [ ]:
class VideoRecordCallback(BaseCallback):
    def __init__(
        self,
        save_path: str,
        video_length: int,
        save_freq: int = 5_000,
        name_prefix: str ="rl_model",
        verbose: int = 0):

        super().__init__(verbose)
        self.save_freq = save_freq
        self.video_length = video_length
        self.save_path = save_path
        self.name_prefix = name_prefix
        # Those variables will be accessible in the callback
        # (they are defined in the base class)
        # The RL model
        # self.model = None  # type: BaseAlgorithm
        # An alias for self.model.get_env(), the environment used for training
        # self.training_env # type: VecEnv
        # Number of time the callback was called
        # self.n_calls = 0  # type: int
        # num_timesteps = n_envs * n times env.step() was called
        # self.num_timesteps = 0  # type: int
        # local and global variables
        # self.locals = {}  # type: Dict[str, Any]
        # self.globals = {}  # type: Dict[str, Any]
        # The logger object, used to report things in the terminal
        # self.logger # type: stable_baselines3.common.logger.Logger
        # Sometimes, for event callback, it is useful
        # to have access to the parent object
        # self.parent = None  # type: Optional[BaseCallback]

    def _on_step(self) -> bool:
        if self.n_calls % self.save_freq == 0:

          name_prefix = f"{self.name_prefix}_{self.num_timesteps}"

          # Record video of the best model playing Atari's Tennis
          rec_val = make_atari_env(
              env_str,
              n_envs=1,
              seed=1,
              wrapper_kwargs=env_val_wrap_args)
          rec_val = VecTransposeImage(rec_val)
          rec_val = VecFrameStack(rec_val, n_stack=4)
          rec_val = VecVideoRecorder(rec_val,
                                    self.save_path,
                                    video_length=self.video_length,
                                    record_video_trigger=lambda x: x == 0,
                                    name_prefix=name_prefix)

          obs = rec_val.reset()
          for _ in range(self.video_length):
              action, _states = self.model.predict(obs)
              obs, rewards, dones, info = rec_val.step(action)
              rec_val.render()
              if dones:
                break

          rec_val.close()
        return True

In [ ]:
# Create Environments
# Create the Training Atari Tennis environment with appropriate wrappers
env = make_atari_env(env_str, n_envs=4)
env = VecTransposeImage(env)
env = VecFrameStack(env, n_stack=4)

# Create the Evaluation Atari Tennis environment with appropriate wrappers
# Disable Frame Skip
env_val = make_atari_env(
    env_str,
    n_envs=1,
    seed=1,
    wrapper_kwargs=env_val_wrap_args)
env_val = VecTransposeImage(env_val)
env_val = VecFrameStack(env_val, n_stack=4)

In [ ]:
# Create Callbacks
# Create Evaluation Callback
# eval_freq - can cause learning instability if set to low
eval_callback = EvalCallback(
    env_val,
    best_model_save_path=model_folder_path,
    log_path=model_folder_path,
    eval_freq=5_000,
    render=False,
    deterministic=True,
    n_eval_episodes=5)

# Create Checkpoint Callback
checkpoint_callback = CheckpointCallback(
    save_freq=5_000,
    save_path=os.path.join(model_folder_path, "checkpoints"),
    name_prefix="atari_tennis_dqn",
    save_replay_buffer=False,
    save_vecnormalize=False,
)

video_record_callback = VideoRecordCallback(
    save_path=os.path.join(model_folder_path, "videos"),
    video_length=2_500,
    save_freq=5_000,
    name_prefix="atari_tennis_dqn")

# Create the callback list
callbackList = CallbackList([checkpoint_callback, video_record_callback, eval_callback])

In [ ]:
# Initialize DQN
model = DQN('CnnPolicy', env,
            verbose=0,
            batch_size=128,
            buffer_size=750_000,
            tensorboard_log=os.path.join(log_dir, "tensorboard"))

# Train the model
model.learn(total_timesteps=10_000_000,
            progress_bar=False,
            callback=callbackList)

# Save the trained model
model.save(os.path.join(model_folder_path, trained_model))

env.close()
env_val.close()

/usr/local/lib/python3.10/dist-packages/gymnasium/utils/passive_env_checker.py:335: UserWarning: WARN: No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps.
  logger.warn(


Moviepy - Building video /content/gdrive/MyDrive/Finding Theta/logs/ALE/Tennis-v5/DQN/training jobs/2024-09-27 18:06:23/videos/atari_tennis_dqn_20000-step-0-to-step-2500.mp4.
Moviepy - Writing video /content/gdrive/MyDrive/Finding Theta/logs/ALE/Tennis-v5/DQN/training jobs/2024-09-27 18:06:23/videos/atari_tennis_dqn_20000-step-0-to-step-2500.mp4



Moviepy - Done !
Moviepy - video ready /content/gdrive/MyDrive/Finding Theta/logs/ALE/Tennis-v5/DQN/training jobs/2024-09-27 18:06:23/videos/atari_tennis_dqn_20000-step-0-to-step-2500.mp4
Eval num_timesteps=20000, episode_reward=-24.00 +/- 0.00
Episode length: 1530.80 +/- 9.97
New best mean reward!


/usr/local/lib/python3.10/dist-packages/gymnasium/utils/passive_env_checker.py:335: UserWarning: WARN: No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps.
  logger.warn(


Moviepy - Building video /content/gdrive/MyDrive/Finding Theta/logs/ALE/Tennis-v5/DQN/training jobs/2024-09-27 18:06:23/videos/atari_tennis_dqn_40000-step-0-to-step-2500.mp4.
Moviepy - Writing video /content/gdrive/MyDrive/Finding Theta/logs/ALE/Tennis-v5/DQN/training jobs/2024-09-27 18:06:23/videos/atari_tennis_dqn_40000-step-0-to-step-2500.mp4



...

Moviepy - Done !
Moviepy - video ready /content/gdrive/MyDrive/Finding Theta/logs/ALE/Tennis-v5/DQN/training jobs/2024-09-27 18:06:23/videos/atari_tennis_dqn_2180000-step-0-to-step-2500.mp4
Eval num_timesteps=2180000, episode_reward=-0.60 +/- 0.80
Episode length: 27000.00 +/- 0.00
Saving video to /content/gdrive/MyDrive/Finding Theta/logs/ALE/Tennis-v5/DQN/training jobs/2024-09-27 18:06:23/videos/atari_tennis_dqn_2200000-step-0-to-step-2500.mp4
Moviepy - Building video /content/gdrive/MyDrive/Finding Theta/logs/ALE/Tennis-v5/DQN/training jobs/2024-09-27 18:06:23/videos/atari_tennis_dqn_2200000-step-0-to-step-2500.mp4.
Moviepy - Writing video /content/gdrive/MyDrive/Finding Theta/logs/ALE/Tennis-v5/DQN/training jobs/2024-09-27 18:06:23/videos/atari_tennis_dqn_2200000-step-0-to-step-2500.mp4



Moviepy - Done !
Moviepy - video ready /content/gdrive/MyDrive/Finding Theta/logs/ALE/Tennis-v5/DQN/training jobs/2024-09-27 18:06:23/videos/atari_tennis_dqn_2200000-step-0-to-step-2500.mp4
Eval num_timesteps=2200000, episode_reward=-1.20 +/- 0.40
Episode length: 27000.00 +/- 0.00
Saving video to /content/gdrive/MyDrive/Finding Theta/logs/ALE/Tennis-v5/DQN/training jobs/2024-09-27 18:06:23/videos/atari_tennis_dqn_2220000-step-0-to-step-2500.mp4
Moviepy - Building video /content/gdrive/MyDrive/Finding Theta/logs/ALE/Tennis-v5/DQN/training jobs/2024-09-27 18:06:23/videos/atari_tennis_dqn_2220000-step-0-to-step-2500.mp4.
Moviepy - Writing video /content/gdrive/MyDrive/Finding Theta/logs/ALE/Tennis-v5/DQN/training jobs/2024-09-27 18:06:23/videos/atari_tennis_dqn_2220000-step-0-to-step-2500.mp4



Moviepy - Done !
Moviepy - video ready /content/gdrive/MyDrive/Finding Theta/logs/ALE/Tennis-v5/DQN/training jobs/2024-09-27 18:06:23/videos/atari_tennis_dqn_2220000-step-0-to-step-2500.mp4
Eval num_timesteps=2220000, episode_reward=-1.20 +/- 0.40
Episode length: 27000.00 +/- 0.00
Saving video to /content/gdrive/MyDrive/Finding Theta/logs/ALE/Tennis-v5/DQN/training jobs/2024-09-27 18:06:23/videos/atari_tennis_dqn_2240000-step-0-to-step-2500.mp4
Moviepy - Building video /content/gdrive/MyDrive/Finding Theta/logs/ALE/Tennis-v5/DQN/training jobs/2024-09-27 18:06:23/videos/atari_tennis_dqn_2240000-step-0-to-step-2500.mp4.
Moviepy - Writing video /content/gdrive/MyDrive/Finding Theta/logs/ALE/Tennis-v5/DQN/training jobs/2024-09-27 18:06:23/videos/atari_tennis_dqn_2240000-step-0-to-step-2500.mp4



Moviepy - Done !
Moviepy - video ready /content/gdrive/MyDrive/Finding Theta/logs/ALE/Tennis-v5/DQN/training jobs/2024-09-27 18:06:23/videos/atari_tennis_dqn_2240000-step-0-to-step-2500.mp4
Eval num_timesteps=2240000, episode_reward=-1.00 +/- 0.00
Episode length: 27000.00 +/- 0.00
Saving video to /content/gdrive/MyDrive/Finding Theta/logs/ALE/Tennis-v5/DQN/training jobs/2024-09-27 18:06:23/videos/atari_tennis_dqn_2260000-step-0-to-step-2500.mp4
Moviepy - Building video /content/gdrive/MyDrive/Finding Theta/logs/ALE/Tennis-v5/DQN/training jobs/2024-09-27 18:06:23/videos/atari_tennis_dqn_2260000-step-0-to-step-2500.mp4.
Moviepy - Writing video /content/gdrive/MyDrive/Finding Theta/logs/ALE/Tennis-v5/DQN/training jobs/2024-09-27 18:06:23/videos/atari_tennis_dqn_2260000-step-0-to-step-2500.mp4



Moviepy - Done !
Moviepy - video ready /content/gdrive/MyDrive/Finding Theta/logs/ALE/Tennis-v5/DQN/training jobs/2024-09-27 18:06:23/videos/atari_tennis_dqn_2260000-step-0-to-step-2500.mp4


In [ ]:
# # Get model path from last training job (uncomment if training job interrupted)
# # List all entries in the directory
# entries = os.listdir(training_data_path)

# # Filter out only directories
# folders = [entry for entry in entries if os.path.isdir(os.path.join(training_data_path, entry))]

# # Sort the folders alphabetically
# folders.sort()

# # Get the last folder
# model_folder_path = os.path.join(training_data_path, folders[-1])
# print(model_folder_path)

In [ ]:
# Create the Evaluation Atari Tennis environment with appropriate wrappers
# Disable Frame Skip
env_val = make_atari_env(env_str,
                         n_envs=1,
                         seed=1,
                         wrapper_kwargs=env_val_wrap_args)
env_val = VecTransposeImage(env_val)
env_val = VecFrameStack(env_val, n_stack=4)

# Load the best model
best_model_path = os.path.join(model_folder_path, "best_model")
best_model = DQN.load(best_model_path, env=env_val)

mean_reward, std_reward = evaluate_policy(
    best_model,
    env_val,
    n_eval_episodes=5)
print(f"Best Model - Mean reward: {mean_reward:.2f} +/- {std_reward:.2f}")

best_metrics_path = os.path.join(log_dir, "best_model_metrics.csv")

# Create Best Model Metrics file if not there
if(not os.path.isfile(best_metrics_path)):
  with open(best_metrics_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["run_date",
                     "batch_size",
                     "learning_rate",
                     "num_timesteps",
                     "mean_reward",
                     "std_reward",
                     "n_envs",
                     "gamma",
                     "seed"])

new_data = [os.path.basename(os.path.normpath(model_folder_path)),
            best_model.batch_size,
            best_model.learning_rate,
            best_model.num_timesteps,
            mean_reward,
            std_reward,
            best_model.n_envs,
            best_model.gamma,
            best_model.seed]

with open(best_metrics_path, 'a', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(new_data)

# Record video of the best model playing Atari's Tennis
rec_val = VecVideoRecorder(env_val,
                           os.path.join(model_folder_path, "videos"),
                           video_length=5000,
                           record_video_trigger=lambda x: x == 0,
                           name_prefix="best_model_atari_tennis_dqn")

obs = rec_val.reset()
for _ in range(5000):
    action, _states = best_model.predict(obs)
    obs, rewards, dones, info = rec_val.step(action)
    rec_val.render()
    if dones:
      break

env_val.close()
rec_val.close()

In [ ]:
# Print Model
print(best_model.policy)

In [ ]:
# Load the evaluations.npz file
data = numpy.load(os.path.join(model_folder_path, "evaluations.npz"))

# Extract the relevant data
timesteps = data['timesteps']
results = data['results']

# Calculate the mean and standard deviation of the results
mean_results = numpy.mean(results, axis=1)
std_results = numpy.std(results, axis=1)

# Plot the results
matplotlib.pyplot.figure()
matplotlib.pyplot.plot(timesteps, mean_results)
matplotlib.pyplot.fill_between(timesteps,
                               mean_results - std_results,
                               mean_results + std_results,
                               alpha=0.3)

matplotlib.pyplot.xlabel('Timesteps')
matplotlib.pyplot.ylabel('Mean Reward')
matplotlib.pyplot.title(f"{rl_type} Performance on {env_str}")
matplotlib.pyplot.show()